### A Simple Portfolio Allocation Problem

This example develops a very simple portfolio allocation problem. Real world portfolio allocation is complex!  A good exercise is reviewing this model (after we build it) and testing your modeling skills:  What aspects of the problem are not being modeled?  Are we modeling all the important aspects?  How much do you trust the solutions?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum

### Loading Data

I did the work of reading in the data for you so that you guys can focus on modeling.  Please read the code below and try to understand what it's doing. Ask questions!

In [2]:
#read monthly_ret_simple.csv using numpy, but ignore the first row and ignore first column
monthly_returns = np.genfromtxt('monthly_ret_simple.csv', delimiter=',', skip_header=True)[:,1:]  

#read in just the first row of monthly_ret_simple.csv, ignore first column and label as tickers
tickers = np.genfromtxt('monthly_ret_simple.csv', delimiter=',', max_rows=1, dtype=str) [1:]

#read in just the first column of monthly_ret_simple.csv, ignoring the first row and label as dates
dates = np.genfromtxt('monthly_ret_simple.csv', delimiter=',', skip_header=True, usecols=0, dtype=str)

print(tickers)
print(dates)

#convert monthly_returns into a dictionary where
# the keys are pairs of (date, ticker) and values are the return
monthly_returns_dict = { (dates[i], tickers[j]) : monthly_returns[i,j] for i in range(len(dates)) for j in range(len(tickers)) }


['AAPL' 'ARKK' 'BABA' 'BITO' 'EEM' 'EWJ' 'FSLR' 'GLD' 'GRN' 'HASI' 'ICLN'
 'LIT' 'MSFT' 'NVDA' 'PLD' 'SWBI' 'TSLA' 'TSM' 'USO' 'VNO' 'VOO' 'XOM']
['20211201' '20220101' '20220201' '20220301' '20220401' '20220501'
 '20220601' '20220701' '20220801' '20220901' '20221001' '20221101'
 '20221201' '20230101' '20230201' '20230301' '20230401' '20230501'
 '20230601' '20230701' '20230801' '20230901' '20231001' '20231101'
 '20231201' '20240101']



"tickers", "dates" and "monthly_returns_dict" are goign to be our data in this problem. You'll only need to access those three from now on.

### Test your understanding.  

Answer the following questions about the data.

1. How many months of data do we have? 
2. How many assets are there?
3. Compute a dictionary (keys are tickers) with values equal to the average_return of the coresponding asset in the dataset.

Min average return:  -0.025713755364530057
Max average return:  0.039998657444333036
Median average return:  0.00044921934693615477


## On to the Actual Modeling!

### What happens if you ask for a target return of 10%?

Does this make sense?  what's the largest target return you can ask for?  You can either reason about this, or modify our optimization problem to compute it!  What's the minimal return you could ask for?

### Pareto Curve


There should be a tradeoff between risk and reward.  The larger the target_reward I require, the more risk I must have to accept.  In the next part of the problem, we're looking ot make a plot with the x-axis as the target_reward, and the y-axis as the risk.

### How does this connect to Sensitivity analysis?
Pick a value on the plot, say target_reward = 0.025.  Compute the sensitivity analysis table at that point.  Can you use this table to determine the slope of your plot at target_reward = 0.25?  Where do you expect the next kink is?